In [1]:
#Author: @michaelbrink
#Org: BalloonBox Inc.

In [2]:
from bs4 import BeautifulSoup
import requests
import json
from html.parser import HTMLParser
import urllib.request
import string
import random
import re
import pandas as pd

In [3]:
#Need a JS enabled web service
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import selenium.webdriver.chrome.service as service
from selenium.webdriver.chrome.options import Options
import time

In [4]:
options = Options()

In [5]:
options.add_argument("--disable-notifications")

In [6]:
driver = webdriver.Chrome('/Users/hongbinlin/Downloads/chromedriver',options=options)

In [7]:
# URL = "https://fortune.com/fortune500/2020/search/"
# driver.get(URL)
# content = BeautifulSoup(driver.page_source, 'html.parser')
# time.sleep(10) #bypass cloudflare

# Helper Functions

In [8]:
def ceoFounder():
    driver.find_element_by_xpath(
        "//div[@class='input__inputWrapper--3SQdx searchFilters__filter--3igRc input__input--34yMC input__checkbox--1434-']/label[@for='ceofounder']").click()
    time.sleep(15)

In [9]:
def femaleCEO():
    driver.find_element_by_xpath(
        "//div[@class='input__inputWrapper--3SQdx searchFilters__filter--3igRc input__input--34yMC input__checkbox--1434-']/label[@for='ceowoman']").click()
    time.sleep(15)

In [10]:
def pageTurner():
    driver.find_element_by_xpath("//div[@class='-next']").click()
    time.sleep(15)

## Scrape rows of companies data

In [11]:
def contentScraper():
    content = BeautifulSoup(driver.page_source, 'html.parser')
    example = content.find_all('a', attrs={'class':'searchResults__cellWrapper--39MAj'})
    example = [item.string for item in example] #return a list of all singular elements
    example = [example[i:i + 11] for i in range(0, len(example), 11)]
    return(example)

## Creating a nested frame of data

## Loop to scrape

In [26]:
 def web_scraping(func=0):
    #date range = 2017 - 2020
    #row range = 10 - 100
    #page range = depends on how many pages it has
    # func = ceoFounder or femaleCEO or empty

    outputContent = []

    ## Pilot model
    for year in range(2017, 2021): 
        
        URL = "https://fortune.com/fortune500/"+str(year)+"/search/"
        driver.get(URL)
        time.sleep(15)
        print ("Year " + str(year) + " active!")
        
        # Check if it has the filter
        try:
            driver.find_element_by_xpath("//label[@for='ceowoman']")
            check_bool = True
        except:
            check_bool = False
        print(check_bool)
        print(func!=0)
        if (not check_bool) and (func!=0):
            print('Skip this year')
            continue
        else:
            print('Continue this year')
            pass

        # Set any desired filters
        if func!=0:
            func()
            print ("Filter Actived!")
        
        # expand rows to maximum
        driver.find_element_by_xpath("//select[@aria-label='rows per page']/option[@value='100']").click()
        print ("Rows extended")
        time.sleep(15)

        # Locate the number of page
        pages  = driver.find_element_by_xpath("//span[@class='-pageInfo']/span[@class='-totalPages']").text
        pages = int(pages)
        print("The total number of pages is: ",pages)
        
        for page in range (1, pages+1):

            outputContent.append((year, page, contentScraper()))
            print("Page " + str(page) + " completed")

            #increment/update the page
            pageTurner()
            
        print ("Year " + str(year) + " completed")

    return(outputContent)

In [27]:
content = web_scraping()

Year 2017 active!
False
False
Rows extended
The total number of pages is:  10
Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed
Page 8 completed
Page 9 completed
Page 10 completed
Year 2017 completed
Year 2018 active!
True
False
Rows extended
The total number of pages is:  10
Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed
Page 8 completed
Page 9 completed
Page 10 completed
Year 2018 completed
Year 2019 active!
True
False
Rows extended
The total number of pages is:  10
Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed
Page 8 completed
Page 9 completed
Page 10 completed
Year 2019 completed
Year 2020 active!
True
False
Rows extended
The total number of pages is:  10
Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed

In [28]:
len(content)

40

In [32]:
content[10]

(2018,
 1,
 [['1',
   'Walmart',
   '$500,343.00',
   '3%',
   '$9,862.00',
   '-27.7%',
   '$204,522.00',
   '$263,563.10',
   '-',
   '2,300,000',
   '-'],
  ['2',
   'Exxon Mobil',
   '$244,363.00',
   '17.4%',
   '$19,710.00',
   '151.4%',
   '$348,691.00',
   '$316,157.10',
   '2',
   '71,200',
   '2'],
  ['3',
   'Berkshire Hathaway',
   '$242,137.00',
   '8.3%',
   '$44,940.00',
   '86.7%',
   '$702,095.00',
   '$492,008.40',
   '-1',
   '377,000',
   '-1'],
  ['4',
   'Apple',
   '$229,234.00',
   '6.3%',
   '$48,351.00',
   '5.8%',
   '$375,319.00',
   '$851,317.90',
   '-1',
   '123,000',
   '-1'],
  ['5',
   'UnitedHealth Group',
   '$201,159.00',
   '8.8%',
   '$10,558.00',
   '50.5%',
   '$139,058.00',
   '$207,079.90',
   '1',
   '260,000',
   '1'],
  ['6',
   'McKesson',
   '$198,533.00',
   '3.1%',
   '$5,070.00',
   '124.5%',
   '$60,969.00',
   '$29,067.00',
   '-1',
   '64,500',
   '-1'],
  ['7',
   'CVS Health',
   '$184,765.00',
   '4.1%',
   '$6,622.00',
   '24.5%

In [ ]:
#outputContent[x][y][z]
#x ranges from 20 - 59 (30 pages in total)
#y ranges from 0 -1 (first or second tuple element where first is the year second is the row data)
#z ranges from 0 -99 (100 rows per page in total)

## Convert nested list to dataframe

In [ ]:
import pandas as pd

In [ ]:
temp = []

In [ ]:
columns =['Rank',
             'Name',
             'Revenue ($M)',
             'Revenue % change', 
             'Profit ($M)',
             'Profit % change',
             'Assets ($M)',
             'Market Value ($M)',
             'Employees',
             'Change in rank (1000)',
             'Change in rank (500)',
             'Year']

In [ ]:
df = pd.DataFrame(columns=columns)

In [ ]:
'''##TO DELETE EVERY nth ITEM in a LIST
counter=0
for i in range(0, 40):
    for j in range (0, 100):
        del outputContent[i][1][j][11-1::11]
        counter += 1'''

In [ ]:
counter=0
for i in range(0, 40):
    for j in range (0, 100):
        df.loc[counter] = outputContent[i][1][j][0:12]
        counter += 1

In [ ]:
#df.to_excel("forbes2017-2020.xlsx")  

# Import saved data file

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('forbes2017-2020.xlsx')

## Data Cleaning

In [ ]:
df.iloc[1001:1300].head()

In [ ]:
import copy #don't point, make a copy!

In [ ]:
temp_emp = copy.copy(df[1000:]['Change in rank (1000)'])

In [ ]:
temp_change1000 = copy.copy(df[1000:]['Employees'])

In [ ]:
df[1000:]['Employees'] = temp_emp
df[1000:]['Change in rank (1000)'] = temp_change1000

In [ ]:
df['Revenue % change'] = df['Revenue % change'].replace('[\%,]', '', regex=True)
df['Revenue % change'] = pd.to_numeric(df['Revenue % change'],errors='coerce').fillna('')

In [ ]:
df[df.columns[4]] = df[df.columns[4]].replace('[\$,]', '', regex=True)
df[df.columns[4]] = pd.to_numeric(df[df.columns[4]],errors='coerce').fillna('')

df[df.columns[7]] = df[df.columns[7]].replace('[\$,]', '', regex=True)
df[df.columns[7]] = pd.to_numeric(df[df.columns[7]],errors='coerce').fillna('')

In [ ]:
#currency fields
df[df.columns[2]] = df[df.columns[2]].replace('[\$,]', '', regex=True).astype(float)
#df[df.columns[4]] = df[df.columns[4]].replace('[\$,]', '', regex=True).astype(float) #issue
df[df.columns[6]] = df[df.columns[6]].replace('[\$,]', '', regex=True).astype(float)
#df[df.columns[7]] = df[df.columns[7]].replace('[\$,]', '', regex=True).astype(float) #issue

In [ ]:
df[df['Year'] == 2018].head(3)

In [ ]:
df['Rank'] = df['Rank'].replace('[,]', '', regex=True)


In [ ]:
#employees
df['Employees'] = df['Employees'].replace('[,]', '', regex=True)
df['Employees'] = pd.to_numeric(df['Employees'],errors='coerce').fillna('')

In [ ]:
#Revenue changes
df['Revenue % change'] = df['Revenue % change'].replace('[\%,]', '', regex=True)
df['Revenue % change'] = pd.to_numeric(df['Revenue % change'],errors='coerce').fillna('')

df['Profit % change'] = df['Profit % change'].replace('[\%,]', '', regex=True)
df['Profit % change'] = pd.to_numeric(df['Profit % change'],errors='coerce').fillna('')

In [ ]:
#Change in rank
df['Change in rank (1000)'] = pd.to_numeric(df['Change in rank (1000)'],errors='coerce').fillna('0')
df['Change in rank (500)'] = pd.to_numeric(df['Change in rank (500)'],errors='coerce').fillna('0')

In [ ]:
df['Profit ($M)'] = pd.to_numeric(df['Profit ($M)'])

In [ ]:
df['Market Value ($M)'] = pd.to_numeric(df['Market Value ($M)'])

In [ ]:
df['Rank'] = pd.to_numeric(df['Rank'])

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
df.info()

In [ ]:
sns.catplot(x="Market Value ($M)", y="Assets ($M)",kind='swarm', data=df[df["Year"]==2020], aspect=1);

In [ ]:
sns.catplot(x="Revenue ($M)", y="Assets ($M)",kind='swarm', data=df[df["Year"]==2020], aspect=1);

# Modelling prep

In [ ]:
df.head(2)

In [ ]:
df.describe()

In [ ]:
dfPlot = df[(df['Year']== 2019) & (df['Employees'] < 80000) & (df['Profit ($M)'] > -10000)]

In [ ]:
dfPlot = dfPlot.dropna()

In [ ]:
dfPlot.shape

In [ ]:
sns.kdeplot( dfPlot['Profit ($M)'], dfPlot['Employees'],
                 cmap="plasma", shade=True, shade_lowest=False)

In [ ]:
sns.kdeplot( dfPlot['Profit ($M)'], dfPlot['Assets ($M)'],
                 cmap="plasma", shade=True, shade_lowest=False)

In [ ]:
sns.kdeplot( dfPlot['Revenue ($M)'], dfPlot['Market Value ($M)'],
                 cmap="plasma", shade=True, shade_lowest=False)

## Support Vector Machine

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X.describe()

In [ ]:
X = dfPlot.drop(['Rank', 'Name', 'Revenue % change','Profit % change', 'Change in rank (1000)', 'Change in rank (500)', 'Year', 'Assets ($M)'],axis=1)
y = dfPlot['Rank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
predictions = lm.predict( X_test)

In [ ]:
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
from sklearn import metrics
import numpy as np

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.distplot((y_test-predictions),bins=50);

In [ ]:
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients